In [3]:
#운석 무리를 피해서 어디까지 진행하는지를 겨루는 게임입니다. 모든 이미지는 2D로 사용하고 있습니다.



import sys
from math import floor
import pygame
from random import randint
from pygame.locals import QUIT,KEYDOWN,KEYUP,K_LEFT,K_RIGHT,K_UP,K_DOWN


pygame.init()
SURFACE = pygame.display.set_mode((800,800))
FPSCLOCK = pygame.time.Clock()

def main():

    ship = [0,0]
    keymap= []
    stars=[]
    scope_image = pygame.image.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\2_images\scope.png")
    rock_image = pygame.image.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\2_images\rock.png")
    while len(stars) < 200:
        stars.append({"pos":[randint(-1600,1600),randint(-1600,1600),randint(0,4095)],"theta":randint(0,360)})

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if not event.key in keymap:
                    keymap.append(event.key)
            elif event.type == KEYUP:
                keymap.remove(event.key)

        # 그리기
        SURFACE.fill((0,0,0))
        
        stars = sorted(stars, key=lambda x: x["pos"][2], reverse=True)
            #화면에 별을 그릴 때, 멀리 있는 것부터 그리기 위해 z좌표 기준으로 정렬”하는 코드일 가능성이 높아
            #stars: 리스트야. 각 원소는 딕셔너리일 가능성이 높고, "pos"라는 키를 가지고 있어.
            #각 별(x)에서 "pos"라는 리스트의 **세 번째 값(index 2)**을 꺼내서 그것으로 비교하겠다는 뜻이

        for star in stars:
            zpos = star["pos"][2]
            xpos= ((star["pos"][0] - ship[0]) << 9) / zpos +400
                #“별의 실제 x위치와 우주선의 x위치 차이를 계산해서,z깊이에 따라 축소하고, 화면 중앙 기준으로 이동시켜화면에 보이는 별의 좌표를 구한다
                # <<9 는 비트 왼쪽 시프트 연산자야   a << 9는 a * 2^9, 즉 a * 512와 같아.
            ypos= ((star["pos"][1] - ship[1]) << 9) / zpos +400
            size = (50 << 9) / zpos
            rotated=pygame.transform.rotozoom(rock_image,star["theta"],size/145)
            SURFACE.blit(rotated,(xpos,ypos))
            
        pygame.display.update()
        FPSCLOCK.tick(20)

if __name__ =='__main__':
    main()

SystemExit: 

In [5]:
#운석 무리를 피해서 어디까지 진행하는지를 겨루는 게임입니다. 모든 이미지는 2D로 사용하고 있습니다.



import sys
from math import floor
import pygame
import time
from random import randint
from pygame.locals import QUIT,KEYDOWN,KEYUP,K_LEFT,K_RIGHT,K_UP,K_DOWN


pygame.init()
SURFACE = pygame.display.set_mode((800,800))
FPSCLOCK = pygame.time.Clock()
pygame.mixer.music.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\Escape.mp3")
pygame.mixer.music.play()

hit_sound = pygame.mixer.Sound(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\bang.wav")   # 짧은 wav/mp3 파일
def main():

    game_over = False
    ship = [0,0]
    keymap= []
    stars=[]
    scope_image = pygame.image.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\2_images\scope.png")
    rock_image = pygame.image.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\2_images\rock.png")
    score= 0
    speed = 25
    scorefont= pygame.font.SysFont(None,36)                           
    sysfont= pygame.font.SysFont(None,72)                           
    message_over= sysfont.render("GAME OVER!!",True,(0,255,255))
    message_rect = message_over.get_rect()
    message_rect.center =(400,400)

    while len(stars) < 200:
        stars.append({"pos":[randint(-1600,1600),randint(-1600,1600),randint(0,4095)],"theta":randint(0,360)})
        # 겉으로 보면 (800,800)짜리 화면인데 좌표는 (-1600,1600)이라니, 너무 커 보이지.하지만 이건 카메라(혹은 우주선) 기준 좌표계를 먼저 잡고, 그걸 화면에 투영할 때 축소해서 그려주는 방식이라 그렇다.
        #zpos로 나눠주니까,거리가 멀수록(즉 zpos가 클수록) 화면에서는 작게 이동돼서 보이게 돼. 즉 (-1600,1600)이라는 큰 좌표도 화면에 찍힐 때는 z에 의해 자동으로 축소 투영되는 거지.
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if not event.key in keymap:
                    keymap.append(event.key)
            elif event.type == KEYUP:
                keymap.remove(event.key)
        
        # 프레임별처리
        if not game_over:
            score += 1
            if score % 10 == 0:
                speed += 1

            if K_LEFT in keymap:
                ship[0] -= 30
            if K_RIGHT in keymap:
                ship[0] += 30
            if K_UP in keymap:
                ship[1] -= 30
            if K_DOWN in keymap:
                ship[1] += 30

            ship[0] = max(-800,min(800,ship[0]))
            ship[1] = max(-800,min(800,ship[1]))
                #if ship[0] < -800: ship[0] = -800
                #   elif ship[0] > 800: ship[0] = 800'  이렇게 -800 ~ 800사이에 묶어두는 것을 한줄로 표현한것임
            for star in stars:
                star["pos"][2] -= speed
                if star["pos"][2] < 64:
                    if abs(star["pos"][0]-ship[0]) <50  and abs(star["pos"][1]-ship[1]) < 50:
                        game_over = True
                    star["pos"] = [randint(-1600,1600),randint(-1600,1600),4095]
        # 그리기
        SURFACE.fill((0,0,0))
        
        stars = sorted(stars, key=lambda x: x["pos"][2],reverse=True )
            #화면에 별을 그릴 때, 멀리 있는 것부터 그리기 위해 z좌표 기준으로 정렬”하는 코드일 가능성이 높아
            #stars: 리스트야. 각 원소는 딕셔너리일 가능성이 높고, "pos"라는 키를 가지고 있어.
            #각 별(x)에서 "pos"라는 리스트의 **세 번째 값(index 2)**을 꺼내서 그것으로 비교하겠다는 뜻이

        for star in stars:
            zpos = star["pos"][2]
            xpos= ((star["pos"][0] - ship[0]) << 9) / zpos +400
                #“별의 실제 x위치와 우주선의 x위치 차이를 계산해서,z깊이에 따라 축소하고, 화면 중앙 기준으로 이동시켜화면에 보이는 별의 좌표를 구한다
                # <<9 는 비트 왼쪽 시프트 연산자야   a << 9는 a * 2^9, 즉 a * 512와 같아.
            ypos= ((star["pos"][1] - ship[1]) << 9) / zpos +400
            size = (50 << 9) / zpos
            rotated=pygame.transform.rotozoom(rock_image,star["theta"],size/145)
            SURFACE.blit(rotated,(xpos,ypos))

        SURFACE.blit(scope_image,(0,0))
        
        if game_over:

            hit_sound.play()
            SURFACE.blit(message_over,message_rect)
            pygame.mixer.music.stop()
            time.sleep(3)
            pygame.quit()
            sys.exit()

        # 점수 나타내기
        score_str= str(score).zfill(6)  #즉 zfill(6)은 전체 길이를 6자리로 맞추겠다는 뜻이고, 부족한 자리만큼 왼쪽에 0을 채워 넣는 거지.
        score_image = scorefont.render(score_str,True,(0,255,0))
        SURFACE.blit(score_image,(700,50))

        # ship위치표시
        ship_posx= str(ship[0]).zfill(3)  #즉 zfill(6)은 전체 길이를 6자리로 맞추겠다는 뜻이고, 부족한 자리만큼 왼쪽에 0을 채워 넣는 거지.
        ship_posx_image = scorefont.render(ship_posx,True,(0,255,0))
        SURFACE.blit(ship_posx_image,(100,350))

        ship_posy= str(ship[1]).zfill(3)  #즉 zfill(6)은 전체 길이를 6자리로 맞추겠다는 뜻이고, 부족한 자리만큼 왼쪽에 0을 채워 넣는 거지.
        ship_posy_image = scorefont.render(ship_posy,True,(0,255,0))
        SURFACE.blit(ship_posy_image,(380,120))

        pygame.display.update()
        FPSCLOCK.tick(20)

if __name__ =='__main__':
    main()

SystemExit: 